In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import glob
import shutil
import csv

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Conv2D, Input, Dense, MaxPool2D, BatchNormalization, GlobalAvgPool2D, Flatten
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# for callback
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

## Objectif
L'objectif de ce code est de parvenir à classer nos images (crées dans le fichier COPD-notebook.ipynb) en utilisant un réseau de neurones convolutif. Pour se faire, on commence par préparer nos données en divisant nos données en données d'entraînement, de validation et de test
## Préparation des données

In [2]:
def split_data(path_to_data, path_to_save_train, path_to_save_val, split_size=0.4) :
    folders = os.listdir(path_to_data) # la liste des dossiers disponible au chemin donné
    for folder in folders :
        full_path = os.path.join(path_to_data, folder) # pour avoir le chemin complet en ajoutant le nom des dossiers
        images_paths = glob.glob(os.path.join(full_path, '*.jpeg')) # ca prend tous les fichiers à l'intérieur du dossier et les télécharge (le join il va a chaque fois ajouter le path du dossier et ajoutant le nom du fichier) ca nous retourne une liste d'images
        x_train, x_val = train_test_split(images_paths, test_size=split_size) # split en train et validation

        for x in x_train : 
            path_to_folder = os.path.join(path_to_save_train, folder) # pour recréer les même dossier que dans le dossier de base
            if not os.path.isdir(path_to_folder) : 
                os.makedirs(path_to_folder) # si il n'existe pas il le crée
            shutil.copy(x, path_to_folder)
        
        for x in x_val : 
            path_to_folder = os.path.join(path_to_save_val, folder) # pour recréer les même dossier que dans le dossier de base
            if not os.path.isdir(path_to_folder) : 
                os.makedirs(path_to_folder) # si il n'existe pas il le crée
            shutil.copy(x, path_to_folder)

Séparation : données d'entrainement / données de validation (60% - 40%)

In [3]:
path_to_data = "./images"
path_to_save_train = "./Training"
path_to_save_val = "./Validation"
path_to_save_test = "./Test"

In [4]:
split_data(path_to_data, path_to_save_train, path_to_save_val)

Séparation : données de validation / données de test (50% - 50%)

In [5]:
def split_val_test(path_to_data, path_to_save_test, split_size=0.5) :
    folders = os.listdir(path_to_data) # la liste des dossiers disponible au chemin donné
    for folder in folders :
        full_path = os.path.join(path_to_data, folder) # pour avoir le chemin complet en ajoutant le nom des dossiers
        images_paths = glob.glob(os.path.join(full_path, '*.jpeg')) # ca prend tous les fichiers à l'intérieur du dossier et les télécharge (le join il va a chaque fois ajouter le path du dossier et ajoutant le nom du fichier) ca nous retourne une liste d'images
        x_train, x_val = train_test_split(images_paths, test_size=split_size) # split en train et validation
        
        for x in x_val : 
            path_to_folder = os.path.join(path_to_save_test, folder) # pour recréer les même dossier que dans le dossier de base
            if not os.path.isdir(path_to_folder) : 
                os.makedirs(path_to_folder) # si il n'existe pas il le crée
            shutil.move(x, path_to_folder)

In [ ]:
split_val_test(path_to_save_val, path_to_save_test)